Hao Wu  July 18 2020 <br />
This code is intended to generate training data for our model

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib
from tqdm.notebook import tqdm
import pandas as pd
import itertools
import statistics

# 1. Preparation

## 1.1 Read & Check Distance Matrices

In [2]:
maps = os.listdir("contact_maps/")
pred = os.listdir("contact_predictions/")
maps_code = [m.split("_")[0] for m in maps]
pred_code = [p.split("_")[0] for p in pred]
# check duplicated elements in maps or preds
print (len(set(maps_code)) == len(maps_code))
print (len(set(pred_code)) == len(pred_code))

# check whether maps & preds match
print (set(maps_code) == set(pred_code)) # True, which means we have same set of acid code for real contact map and predictions
num_of_code = len(set(maps_code))
print (num_of_code) # number of acid codes/contact maps we are analyzing 


True
True
True
11181


## 1.2 Generate Square Diff Matrices 

1. For each pair of contact map & prediction, calculate squared difference between corresponding points of the two matrices and generate a square diff matrix.
2. Collect all data points in the upper triangular part of each sqr-diff matrix and store in a list
* upper triangular part: since data points on each contact map are symmetric about y=x, we can either focus on upper triangular part of the matrix (y>=x) or on lower triangular part (x>=y)

In [3]:
sqr_diff = {}
contact_pred = {}
diff_ele = []
for c in tqdm(maps_code):
    temp_map = np.load("contact_maps/"+c+"_contact_map.npy")
    temp_pred = np.load("contact_predictions/"+c+"_contact_pred.npy")
    temp = np.power(np.subtract(temp_map, temp_pred),2)
    size = temp.shape[0]
    contact_pred[c] = (temp_pred, size)
    sqr_diff[c] = (temp, size)
    iu = np.triu_indices(size) # indices in upper triangle matrix
    
    # concatenating flatten np matrix with list is faster than merging np matrix
    diff_ele += list(temp[iu].flat) 


## 1.3 Hyperparameters

There are two hyperparameters:
1. cutoff of sqr-diff value
2. neighbor range

### Choose Cutoff of Squared Difference Value 

1. The cutoff serves to determine whether a squared difference value is large enough so that the corresponding data point in predicted contact map can be considered as an error.
2. Based on the list of all sqr-diff data points, we can choose a cutoff according to its distribution.
3. Here I have chosen 75% quantile of all the sqr-diff data.

In [4]:
print (len(sqr_diff))
print (len(diff_ele)) # 736777153 | 369656525

# takes about 5-6 | 2-3 min to compute
#cutoff = np.quantile(diff_ele,0.75) # 7.172270573359031e-06 | 7.068872962487293e-06
cutoff = 7.068872962487293e-06
#print (cutoff)

11181
369656525


### Neighbor Range & Get Neighbors Function

Neighbor range determines the size of region we are going to check around certain data point. 
For example, for a data point (i,j), given neighbor range n = 1, the neighbor we are going to check will be (i-1,j-1), (i,j-1), (i+1,j-1), (i-1,j), (i+1,j), (i-1,j+1), (i,j+1), (i+1,j) 

In [5]:
nb_range = 3

In [12]:
def get_neighbors(x,y,n,s,matrix):
    '''
    perm = list(itertools.product(list(range(max(x-n,0),min(x+n+1,s))),list(range(max(y-n,0),min(y+n+1,s)))))
    perm.remove((x,y))
    half_perm_values = [matrix[p[0]][p[1]] for p in perm if p[0]<=p[1]]
    '''
    
    neighbor_matrix = matrix[max(x-n,0):min(x+n+1,s), max(y-n,0):min(y+n+1,s)]
    iu_neighbor_matrix = np.triu_indices(neighbor_matrix.shape[0])
    half_perm_values = list(neighbor_matrix[iu_neighbor_matrix].flat)
    half_perm_values.remove(matrix[x][y])
    return (min(half_perm_values),statistics.median(half_perm_values),max(half_perm_values),statistics.mean(half_perm_values),statistics.stdev(half_perm_values))


In [7]:
 '''
temp_points = [(98, 98), (98, 99)]
temp_try = [tt for t in temp_points for tt in get_neighbors(t[0],t[1],1,100) ]
print (temp_try)
 '''

'\ntemp_points = [(98, 98), (98, 99)]\ntemp_try = [tt for t in temp_points for tt in get_neighbors(t[0],t[1],1,100) ]\nprint (temp_try)\n'

# 2. Generate Training Data

The training data will be in form of feature values + output value. Each row of entries corresponds to a point in the predicted contact map, and includes features about 
1. relative position in the matrix 
2. predicted value 
3. predicted values of adjacent points. 

The output value of each row is either 1 or 0. For each point in the predicted contact map, classify whether the data point is an error based on its value in sqr-diff matrix and cutoff value. If the sqr-diff is larger than cutoff, it will be classified as "TRUE" (output value 1), which means it is large enough to be considered as an error. On the contrary, if the sqr-diff is less than cutoff, it will be classified as "FALSE" (output value 0), which means it's not considered an error.

### Relative Position

Calculate relative position of each data point (relative position = x or y coordinate / size of matrix. e.g. (5,10) in a matrix of size 10 will have relative position (5/10, 10/10) = (0.5, 1))

In [13]:
pos_true = []
pred_value_true = []
neighbor_value_true = []

pos_false = []
pred_value_false = []
neighbor_value_false = []

# takes about 20 min to compute
for m in tqdm(maps_code):
    il = np.tril_indices(sqr_diff[m][1], -1) # index of lower triangle matrix without diagonal
    # change values in the lower triangular part to an arbitrary large number so that they won't be included in our training data
    sqr_diff[m][0][il] = 1000000  
    
    # --- True Data Points ---
    temp_result_true = np.where((sqr_diff[m][0]>=cutoff) & (sqr_diff[m][0]<1000000))
    '''
    # extract relative position of TRUE data points
    temp_result_true_x = [x/sqr_diff[m][1] for x in temp_result_true[0]]
    temp_result_true_y = [y/sqr_diff[m][1] for y in temp_result_true[1]]
    pos_true += list(zip(temp_result_true_x, temp_result_true_y)) # relative position stored in a list
    '''
    
    # get predicted value at (x,y) of predicted contact map m
    pred_value_true = [contact_pred[m][0][temp_result_true[0][i]][temp_result_true[1][i]] for i in range(len(temp_result_true[0]))]
    # TO DO: get predicted value of points adjacent to (x,y) of predicted contact map m
    neighbor_value_true = [get_neighbors(temp_result_true[0][i],temp_result_true[1][i],nb_range,sqr_diff[m][1],contact_pred[m][0]) for i in range(len(temp_result_true[0]))]
    '''
    for i in range(len(temp_result_true[0])):
        temp_neighbor_values = [contact_pred[m][0][g[0]][g[1]] for g in get_neighbors(temp_result_true[0][i],temp_result_true[1][i],nb_range,sqr_diff[m][1])]
        neighbor_value_true += (min(temp_neighbor_values),statistics.median(temp_neighbor_values),max(temp_neighbor_values),statistics.mean(temp_neighbor_values),statistics.stdev(temp_neighbor_values))
    '''
    # --- False Data Points ---
    temp_result_false = np.where(sqr_diff[m][0]<cutoff)
    '''
    # extract relative position of FALSE data points
    temp_result_false_z = [z/sqr_diff[m][1] for z in temp_result_false[0]]
    temp_result_false_w = [w/sqr_diff[m][1] for w in temp_result_false[1]]
    pos_false += list(zip(temp_result_false_z, temp_result_false_w))
    '''
    
    # get predicted value at (x,y) of predicted contact map m
    pred_value_false = [contact_pred[m][0][temp_result_false[0][i]][temp_result_false[1][i]] for i in range(len(temp_result_false[0]))]
    # TO DO: get predicted value of points adjacent to (x,y) of predicted contact map m
    '''
    for i in range(len(temp_result_false[0])):
        temp_neighbor_values = [contact_pred[m][0][g[0]][g[1]] for g in get_neighbors(temp_result_false[0][i],temp_result_false[1][i],nb_range,sqr_diff[m][1])]
        neighbor_value_false += (min(temp_neighbor_values),statistics.median(temp_neighbor_values),max(temp_neighbor_values),statistics.mean(temp_neighbor_values),statistics.stdev(temp_neighbor_values))
    '''
    
    

IndexError: index 4 is out of bounds for axis 1 with size 4

In [ ]:
print (len(pos_true)) # 9214132
print (len(pos_false)) # 277242393
# 9214132 + 277242393 = 369656525, which echoes the number of all data points

# convert list to dataframe
data_true = pd.DataFrame(pos_true,columns=["pos_x", "pos_y"])
data_false = pd.DataFrame(pos_false,columns=["pos_x", "pos_y"])
data_true["output"] = 1 # TRUE data is marked 1
data_false["output"] = 0 # FALSE data is marked 0
print (data_true)
print (data_false)


In [ ]:
# output dataframes of training data to csv files
data_true.to_csv("training_data_true.csv", index=False) # 3.64 GB
data_false.to_csv("training_data_false.csv", index=False) # 10.93 GB

# Visualize Contact Maps

not relevant to getting training data but may be referenced later

In [ ]:
viz_cm = np.load("contact_maps/1A0CA_contact_map.npy")
plt.matshow(viz_cm)
plt.show()

In [ ]:
viz_cp = np.load("contact_predictions/1A0CA_contact_pred.npy")
plt.matshow(viz_cp)
plt.show()

In [ ]:
a = [[1,2,3],[4,5,6]]
temp_ab = [(a[0][i],a[1][i]) for i in range(len(a[0]))]
temp_cd = [x for x in zip(a[0],a[1])]
print (temp_ab)
print (temp_cd)

In [ ]:
b = [1,2,4,4]
print (statistics.mean(b))
print (statistics.median(b))
print (min(b))
print (max(b))
print (statistics.stdev(b))